<a href="https://colab.research.google.com/github/rajabhupati/AI-Session/blob/main/Prompt_Engineering/prompt_engineering_with_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering with Open Source LLMs
This notebook demonstrates zero-shot, few-shot, and instruction prompting using `Mistral-7B-Instruct` on Hugging Face.

*Note: Ollama is not supported in Google Colab as it requires a local Docker runtime.*

In [ ]:
# Install required packages
!pip install -q transformers accelerate bitsandbytes

In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Sat May  3 11:15:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0       

In [ ]:
import warnings
# Suppress specific UserWarnings related to computation
warnings.filterwarnings("ignore", message="Input type into Linear4bit")

In [ ]:
import logging

# Suppress info-level messages in transformers
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
# Replace 'your_token_here' with an actual token temporarily for testing
import os
from google.colab import userdata


# Directly set the Hugging Face token in your code (not ideal for real use due to exposure)
huggingface_token = userdata.get('HUGGINGFACE_TOKEN')

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Ensure token is accessible
if not huggingface_token:
    raise ValueError("HUGGINGFACE_TOKEN is not set. Please ensure it's configured correctly.")

# Load model and tokenizer using token
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    token=huggingface_token
)

def generate(prompt, max_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=True, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

## Zero-Shot Prompting

In [ ]:
prompt = "Translate this sentence to French: 'I love machine learning.'"
print(generate(prompt))

Translate this sentence to French: 'I love machine learning.'

'Je adore l'apprentissage de la machine.'


## Few-Shot Prompting

In [ ]:
def generate_response(prompt, max_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=True, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
few_shot_prompt_grammar = '''Correct the grammatical errors in the following sentences:
Incorrect: The cat don't like water.
Correct: The cat doesn't like water.

Incorrect: She go to store yesterday.
Correct: She went to the store yesterday.

Incorrect: He are playing in the park.
Correct:'''
print(generate_response(few_shot_prompt_grammar))

Correct the grammatical errors in the following sentences:
Incorrect: The cat don't like water.
Correct: The cat doesn't like water.

Incorrect: She go to store yesterday.
Correct: She went to the store yesterday.

Incorrect: He are playing in the park.
Correct: He is playing in the park.

Incorrect: They will be arriving at 5 pm.
Correct: They will be arriving at 5 pm.

Incorrect: I have saw the movie before.
Correct: I have seen the movie before.


## Instruction Prompting

In [ ]:
# Initialize the expanded instruction prompt with the larger paragraph
instruction_prompt = (
    "You are a helpful assistant. Summarize the following paragraph:\n\n"
    "Machine learning is a branch of artificial intelligence that focuses on building systems capable of "
    "learning from data and making predictions or decisions without being explicitly programmed. It encompasses "
    "a variety of techniques, including supervised learning, where models are trained on labeled datasets, and "
    "unsupervised learning, where they identify patterns or structures in data without preexisting labels. "
    "Deep learning, a subset of machine learning, uses neural networks with many layers to model complex "
    "patterns and relationships in data. Machine learning is transforming industries by providing powerful "
    "tools for analyzing large datasets, automating tasks, optimizing processes, and enhancing decision-making. "
    "Applications range from computer vision and natural language processing to recommendation systems and "
    "predictive analytics. As data continues to grow exponentially, the demand for effective machine learning "
    "models increases, driving research into new architectures, training methods, and interpretability to "
    "bridge the gap between theoretical advancements and real-world applications.\n\nSummary:"
)

# Generate the summary using the defined function
print(generate(instruction_prompt))

You are a helpful assistant. Summarize the following paragraph:

Machine learning is a branch of artificial intelligence that focuses on building systems capable of learning from data and making predictions or decisions without being explicitly programmed. It encompasses a variety of techniques, including supervised learning, where models are trained on labeled datasets, and unsupervised learning, where they identify patterns or structures in data without preexisting labels. Deep learning, a subset of machine learning, uses neural networks with many layers to model complex patterns and relationships in data. Machine learning is transforming industries by providing powerful tools for analyzing large datasets, automating tasks, optimizing processes, and enhancing decision-making. Applications range from computer vision and natural language processing to recommendation systems and predictive analytics. As data continues to grow exponentially, the demand for effective machine learning mode

## Prompt Variation (Soft Prompt Tuning)

In [ ]:
tweaked_prompt = "Please act as a French translator. What is the French for: 'I love machine learning'?"
print(generate(tweaked_prompt))

Please act as a French translator. What is the French for: 'I love machine learning'?

I love machine learning

"Je aime l'apprentissage de la machine"
